In [ ]:
#安装tushare扩展shell
pip install tushare
pip install lxml
#版本升级
pip install tushare --upgrade

In [1]:
import tushare

print(tushare.__version__)

1.2.84


In [ ]:
免费注册用户https://waditu.com获取token，注册好后在我的主页里有一个接口TOKEN
import tushare as ts
ts.set_token('换成自己的token')
pro = ts.pro_api()
data = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')

In [ ]:
#获取个股的日线行情数据，就会返回代码为‘000001’的股票在20200608到20200610之间的数据，后面的sz是深圳的缩写，必需要加上。还有一些股票后面是sh，就是上海。
df = pro.daily(ts_code='000001.SZ', start_date='20200608', end_date='20200610')

In [ ]:
#pandas的数据形式把中间部分给省略了，查看df的全部数据用一个命令展开数据
df.unstack()

In [ ]:
df['open']
df['open'][0]
df.iloc[0]
df.iloc[0]['open']
df.iloc[0]['close']

In [ ]:
#抓取所有股票一段时间内股东人数变化并将变化量并进行排序的代码
from pandas import DataFrame
import tushare as ts
import time
ts.set_token('换成自己的token')
pro = ts.pro_api()
stock_info = pro.stock_basic()#获取所有股票的基本信息
#print(len(stock_info))
startdate: str = input('请输入起始时间，格式为20210304\n')
enddate: str = input('请输入结束时间\n')
code: str = input('请输入查询股票的代码，输入0则查询所有股票\n')
variation = {}
if code != '0':
  stockholder_num = pro.stk_holdernumber(ts_code=code,start_date=startdate,end_date=enddate)
  #print(stockholder_num)
  df=DataFrame(stockholder_num)
  df.to_excel('stockholder_num.xlsx')
else:
  for i in range(0,len(stock_info)):#遍历所有股票
    if i>0 and i % 100 == 0:
      time.sleep(60)#由于每分钟调用限制，每调用100次等60s
    code = stock_info.at[i,'ts_code']
    #print(code)
    stockholder_num = pro.stk_holdernumber(ts_code=code,start_date=startdate,end_date=enddate)
    #print(stockholder_num)
    try:#由于一段时间内不一定每只股票都公告了股东人数，所以有可能会报错
      later = stockholder_num.at[0,'holder_num']
      former = stockholder_num.at[len(stockholder_num)-1,'holder_num']
      change = later - former
    except:#如果没有公告股东人数则跳过这一支股票进入下一支
      continue
    #print(change)
    variation[stock_info.at[i,'ts_code']] = change#将股东人数变化量存入字典
    #print(i)

  rank = sorted(variation.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)#给字典排序
  print(rank)
  df=DataFrame(rank)
  df.to_excel('stockholder_num.xlsx')#将数据存入Excel表中
